# Downloading Youtube Video using Python

In [ ]:
!pip install pytubefix
!pip install pydub

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os
from pydub import AudioSegment

# Paths
SAVE_PATH = "/content/videos"  # Ensure this path exists
AUDIO_PATH = "/content/audio"   # Ensure this path exists

# Link of the video to be downloaded
link = "https://www.youtube.com/watch?v=1aA1WGON49E"

# Create directories if they don't exist
os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(AUDIO_PATH, exist_ok=True)

# Download the video
yt = YouTube(link, on_progress_callback=on_progress)
video_title = yt.title

print(f"Downloading: {video_title}")

# Save the video
video_stream = yt.streams.get_highest_resolution()
video_stream.download(output_path=SAVE_PATH)

# Extract audio and save it
audio_stream = yt.streams.get_audio_only()
audio_file_path = audio_stream.download(output_path=AUDIO_PATH)

# Convert to MP3
mp3_file_path = os.path.join(AUDIO_PATH, f"{video_title}.mp3")

# Load the audio file
audio = AudioSegment.from_file(audio_file_path)

# Export as MP3
audio.export(mp3_file_path, format="mp3")

# Optionally, remove the original audio file
os.remove(audio_file_path)

print(f"Audio saved as MP3: {mp3_file_path}")

Downloading: A one minute TEDx Talk for the digital age | Woody Roseland | TEDxMileHigh
Audio saved as MP3: /content/audio/A one minute TEDx Talk for the digital age | Woody Roseland | TEDxMileHigh.mp3




# Converting Audio into Text


In [ ]:
!pip install deepgram-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

# Path to the audio file
AUDIO_FILE = "/content/audio/A one minute TEDx Talk for the digital age | Woody Roseland | TEDxMileHigh.mp3"

API_KEY = userdata.get('Deepgram_ApiKey')


def main():
    try:
        # STEP 1 Create a Deepgram client using the API key
        deepgram = DeepgramClient(API_KEY)

        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        #STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
        transcription = response['results']['channels'][0]['alternatives'][0]['transcript']

        # STEP 4: Print the response
        return transcription
        # print(response.to_json(indent=4))

    except Exception as e:
        print(f"Exception: {e}")


text = main()

<ipython-input-54-0bca07a651c0>:35: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


# Converting Text back to Audio

In [ ]:
!pip install elevenlabs

In [ ]:
from elevenlabs import ElevenLabs, VoiceSettings
from google.colab import userdata

client = ElevenLabs(
    api_key = userdata.get('ElevenLabs_ApiKey'),
)
response = client.text_to_speech.convert(
    voice_id="pMsXgVXv3BLzUgSXRplE",
    optimize_streaming_latency="0",
    output_format="mp3_22050_32",
    text=text,
    voice_settings=VoiceSettings(
        stability=0.1,
        similarity_boost=0.3,
        style=0.2,
    ),
)

# Create the directory if it doesn't exist
output_directory = '/content/generated_audio'
os.makedirs(output_directory, exist_ok=True)

# Save the audio file in the specified directory
output_file_path = os.path.join(output_directory, "output_audio.mp3")
with open(output_file_path, "wb") as audio_file:
    for chunk in response:
        audio_file.write(chunk)

# Comparing Both Audios

In [ ]:
from IPython.display import Audio
print("Original Audio")
Audio(mp3_file_path)

Original Audio


In [ ]:
print("Generated Audio")
Audio(output_file_path)

Generated Audio
